In [1]:
# Cargar el archivo de datos
import pandas as pd

file_path = 'Train_CYC.xlsx'
train_cyc_data = pd.read_excel(file_path)


In [2]:
# Identificar columnas con valores faltantes y sus proporciones
missing_values = train_cyc_data.isnull().sum()
missing_percentage = (missing_values / len(train_cyc_data)) * 100
missing_data = pd.DataFrame({'Missing Values': missing_values, 'Percentage (%)': missing_percentage})
missing_data.sort_values(by='Missing Values', ascending=False)

,Missing Values,Percentage (%)
aspect_ratio,1646,36.577778
content_rating,1631,36.244444
plot_keywords,1541,34.244444
director_name,1516,33.688889
director_facebook_likes,1516,33.688889
num_critic_for_reviews,1478,32.844444
actor_3_facebook_likes,1466,32.577778
actor_3_name,1466,32.577778
num_user_for_reviews,1460,32.444444
actor_2_facebook_likes,1457,32.377778


In [3]:
# Eliminar columnas con más del 30% de datos faltantes
columns_to_drop = missing_data[missing_data['Percentage (%)'] > 30].index
train_cyc_data_cleaned = train_cyc_data.drop(columns=columns_to_drop)
train_cyc_data_cleaned.head()

,Id,movie_title,title_year,color,duration,genres,gross,language,budget,imdb_score,tomato_score,tomato_audience_score,label
0,1,Something Wild,1986.0,Color,113.0,Action|Comedy|Drama,8362969.0,English,0.0,6.5,0.38,0.80,1
1,2,Tiger Orange,2014.0,Color,75.0,Drama,NaN,English,100000.0,6.8,0.76,1.00,0
2,3,Cast Away,2000.0,Color,143.0,Adventure|Drama|Romance,233630478.0,English,90000000.0,7.7,0.17,0.62,1
3,4,The Sisterhood of the Traveling Pants,2005.0,Color,119.0,Comedy|Drama|Family|Romance,39008741.0,English,25000000.0,6.6,0.35,0.93,1
4,5,Fight Valley,2016.0,Color,90.0,Action|Drama,NaN,English,27000000.0,5.0,0.49,0.27,0


In [4]:
# Imputar valores faltantes usando la moda
for column in ['title_year', 'color', 'duration', 'gross', 'language', 'budget']:
    mode_value = train_cyc_data_cleaned[column].mode()[0]
    train_cyc_data_cleaned[column].fillna(mode_value, inplace=True)

train_cyc_data_cleaned.head()

,Id,movie_title,title_year,color,duration,genres,gross,language,budget,imdb_score,tomato_score,tomato_audience_score,label
0,1,Something Wild,1986.0,Color,113.0,Action|Comedy|Drama,8362969.0,English,0.0,6.5,0.38,0.80,1
1,2,Tiger Orange,2014.0,Color,75.0,Drama,2000000.0,English,100000.0,6.8,0.76,1.00,0
2,3,Cast Away,2000.0,Color,143.0,Adventure|Drama|Romance,233630478.0,English,90000000.0,7.7,0.17,0.62,1
3,4,The Sisterhood of the Traveling Pants,2005.0,Color,119.0,Comedy|Drama|Family|Romance,39008741.0,English,25000000.0,6.6,0.35,0.93,1
4,5,Fight Valley,2016.0,Color,90.0,Action|Drama,2000000.0,English,27000000.0,5.0,0.49,0.27,0


In [5]:
# Extraer todas las categorías únicas de la columna 'genres'
unique_genres = set()
train_cyc_data_cleaned['genres'].str.split('|').apply(unique_genres.update)

# Crear columnas binarias para cada categoría de género
for genre in unique_genres:
    train_cyc_data_cleaned[f'genre_{genre}'] = train_cyc_data_cleaned['genres'].apply(lambda x: 1 if genre in x else 0)

# Eliminar la columna original 'genres' si ya no es necesaria
train_cyc_data_cleaned = train_cyc_data_cleaned.drop(columns=['genres'])

# Verifica las primeras filas para confirmar la codificación
train_cyc_data_cleaned.head()


,Id,movie_title,title_year,color,duration,gross,language,budget,imdb_score,tomato_score,...,genre_Short,genre_Drama,genre_Animation,genre_Reality-TV,genre_Comedy,genre_Foreign,genre_War,genre_Sci-Fi,genre_Thriller,genre_Music
0,1,Something Wild,1986.0,Color,113.0,8362969.0,English,0.0,6.5,0.38,...,0,1,0,0,1,0,0,0,0,0
1,2,Tiger Orange,2014.0,Color,75.0,2000000.0,English,100000.0,6.8,0.76,...,0,1,0,0,0,0,0,0,0,0
2,3,Cast Away,2000.0,Color,143.0,233630478.0,English,90000000.0,7.7,0.17,...,0,1,0,0,0,0,0,0,0,0
3,4,The Sisterhood of the Traveling Pants,2005.0,Color,119.0,39008741.0,English,25000000.0,6.6,0.35,...,0,1,0,0,1,0,0,0,0,0
4,5,Fight Valley,2016.0,Color,90.0,2000000.0,English,27000000.0,5.0,0.49,...,0,1,0,0,0,0,0,0,0,0


In [6]:
# Eliminar las columnas especificadas
columns_to_remove = ['Id', 'movie_title', 'color', 'language']
train_cyc_data_cleaned = train_cyc_data_cleaned.drop(columns=columns_to_remove)

# Verificar que las columnas han sido eliminadas
train_cyc_data_cleaned.head()


,title_year,duration,gross,budget,imdb_score,tomato_score,tomato_audience_score,label,genre_Game-Show,genre_Horror,...,genre_Short,genre_Drama,genre_Animation,genre_Reality-TV,genre_Comedy,genre_Foreign,genre_War,genre_Sci-Fi,genre_Thriller,genre_Music
0,1986.0,113.0,8362969.0,0.0,6.5,0.38,0.80,1,0,0,...,0,1,0,0,1,0,0,0,0,0
1,2014.0,75.0,2000000.0,100000.0,6.8,0.76,1.00,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,2000.0,143.0,233630478.0,90000000.0,7.7,0.17,0.62,1,0,0,...,0,1,0,0,0,0,0,0,0,0
3,2005.0,119.0,39008741.0,25000000.0,6.6,0.35,0.93,1,0,0,...,0,1,0,0,1,0,0,0,0,0
4,2016.0,90.0,2000000.0,27000000.0,5.0,0.49,0.27,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [7]:
from sklearn.preprocessing import StandardScaler

# Inicializar el escalador
scaler = StandardScaler()

# Seleccionar las columnas a estandarizar
columns_to_standardize = ['gross', 'budget']

# Estandarizar las columnas y reemplazarlas en el DataFrame
train_cyc_data_cleaned[columns_to_standardize] = scaler.fit_transform(train_cyc_data_cleaned[columns_to_standardize])

# Verificar los primeros datos para confirmar la estandarización
train_cyc_data_cleaned[columns_to_standardize].head()


,gross,budget
0,-0.381981,-0.152148
1,-0.454843,-0.151615
2,2.197555,0.327151
3,-0.031057,-0.019009
4,-0.454843,-0.008358


In [8]:
# Verificar si hay valores faltantes en el DataFrame
missing_values_check = train_cyc_data_cleaned.isnull().sum()

# Filtrar solo las columnas que tienen valores faltantes
missing_values_columns = missing_values_check[missing_values_check > 0]

# Mostrar las columnas con valores faltantes y el número de valores faltantes
if len(missing_values_columns) == 0:
    print("No hay valores faltantes en el conjunto de datos.")
else:
    print("Columnas con valores faltantes:")
    print(missing_values_columns)


No hay valores faltantes en el conjunto de datos.


In [9]:
# Calcular la moda de la columna 'imdb_score'
imdb_score_mode = train_cyc_data_cleaned['imdb_score'].mode()[0]

# Reemplazar los valores faltantes en 'imdb_score' con la moda
train_cyc_data_cleaned['imdb_score'].fillna(imdb_score_mode, inplace=True)

# Verificar si aún quedan valores faltantes en 'imdb_score'
missing_values_imdb_score = train_cyc_data_cleaned['imdb_score'].isnull().sum()

if missing_values_imdb_score == 0:
    print("Todos los valores faltantes en 'imdb_score' han sido reemplazados por la moda.")
else:
    print(f"Aún quedan {missing_values_imdb_score} valores faltantes en 'imdb_score'.")


Todos los valores faltantes en 'imdb_score' han sido reemplazados por la moda.


In [10]:
# Verificar si hay valores faltantes en el DataFrame
missing_values_check = train_cyc_data_cleaned.isnull().sum()

# Filtrar solo las columnas que tienen valores faltantes
missing_values_columns = missing_values_check[missing_values_check > 0]

# Mostrar las columnas con valores faltantes y el número de valores faltantes
if len(missing_values_columns) == 0:
    print("No hay valores faltantes en el conjunto de datos.")
else:
    print("Columnas con valores faltantes:")
    print(missing_values_columns)

No hay valores faltantes en el conjunto de datos.


In [11]:
train_cyc_data_cleaned.head()

,title_year,duration,gross,budget,imdb_score,tomato_score,tomato_audience_score,label,genre_Game-Show,genre_Horror,...,genre_Short,genre_Drama,genre_Animation,genre_Reality-TV,genre_Comedy,genre_Foreign,genre_War,genre_Sci-Fi,genre_Thriller,genre_Music
0,1986.0,113.0,-0.381981,-0.152148,6.5,0.38,0.80,1,0,0,...,0,1,0,0,1,0,0,0,0,0
1,2014.0,75.0,-0.454843,-0.151615,6.8,0.76,1.00,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,2000.0,143.0,2.197555,0.327151,7.7,0.17,0.62,1,0,0,...,0,1,0,0,0,0,0,0,0,0
3,2005.0,119.0,-0.031057,-0.019009,6.6,0.35,0.93,1,0,0,...,0,1,0,0,1,0,0,0,0,0
4,2016.0,90.0,-0.454843,-0.008358,5.0,0.49,0.27,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [12]:
# Calcular la matriz de correlación solo contra la variable target 'label'
correlation_with_label = train_cyc_data_cleaned.corr()['label'].sort_values(ascending=False)

# Mostrar las correlaciones
print(correlation_with_label)


label                    1.000000
imdb_score               0.377721
gross                    0.279612
duration                 0.176661
genre_Drama              0.120239
title_year               0.073496
genre_Biography          0.068987
genre_Sport              0.050022
genre_Documentary        0.030329
genre_Romance            0.017429
genre_Crime              0.015625
tomato_audience_score    0.012981
genre_Mystery            0.008469
tomato_score             0.006939
budget                   0.004508
genre_Sci-Fi            -0.002469
genre_History           -0.003262
genre_Fantasy           -0.004114
genre_TV Movie          -0.007609
genre_Game-Show         -0.007609
genre_News              -0.007609
genre_Music             -0.008783
genre_Reality-TV        -0.010762
genre_Film-Noir         -0.015224
genre_Short             -0.015224
genre_War               -0.015720
genre_Foreign           -0.018649
genre_Science Fiction   -0.019753
genre_Adventure         -0.022361
genre_Western 

In [13]:
from sklearn.feature_selection import mutual_info_classif

# Separar las variables independientes (X) de la variable objetivo (y)
X = train_cyc_data_cleaned.drop(columns=['label'])
y = train_cyc_data_cleaned['label']

# Calcular la información mutua entre las características y la variable objetivo
mutual_info = mutual_info_classif(X, y, discrete_features='auto')

# Crear un DataFrame para visualizar los resultados
mutual_info_df = pd.DataFrame({'Feature': X.columns, 'Mutual Information': mutual_info})

# Ordenar las características por la información mutua
mutual_info_df.sort_values(by='Mutual Information', ascending=False, inplace=True)

# Mostrar los resultados
print(mutual_info_df)


                  Feature  Mutual Information
4              imdb_score            0.168627
2                   gross            0.065558
1                duration            0.031676
0              title_year            0.019501
9         genre_Biography            0.014167
3                  budget            0.014096
27            genre_Drama            0.009500
33           genre_Sci-Fi            0.009330
22      genre_Documentary            0.009204
14          genre_Mystery            0.008551
28        genre_Animation            0.007709
7         genre_Game-Show            0.006804
32              genre_War            0.006461
6   tomato_audience_score            0.005605
13           genre_Family            0.005103
29       genre_Reality-TV            0.003486
16          genre_Fantasy            0.003098
18            genre_Sport            0.002694
8            genre_Horror            0.002316
19            genre_Crime            0.002196
11        genre_Film-Noir         

In [14]:
# Seleccionar las características con información mutua mayor que 0
selected_features = mutual_info_df[mutual_info_df['Mutual Information'] > 0]['Feature'].tolist()

# Crear un nuevo DataFrame con solo las características seleccionadas
train_cyc_data_selected = train_cyc_data_cleaned[selected_features + ['label']]

# Mostrar las primeras filas del nuevo DataFrame para confirmar los cambios
train_cyc_data_selected.head()


,imdb_score,gross,duration,title_year,genre_Biography,budget,genre_Drama,genre_Sci-Fi,genre_Documentary,genre_Mystery,...,genre_Reality-TV,genre_Fantasy,genre_Sport,genre_Horror,genre_Crime,genre_Film-Noir,genre_History,genre_Action,genre_Foreign,label
0,6.5,-0.381981,113.0,1986.0,0,-0.152148,1,0,0,0,...,0,0,0,0,0,0,0,1,0,1
1,6.8,-0.454843,75.0,2014.0,0,-0.151615,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,7.7,2.197555,143.0,2000.0,0,0.327151,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,6.6,-0.031057,119.0,2005.0,0,-0.019009,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,5.0,-0.454843,90.0,2016.0,0,-0.008358,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [15]:
# Exportar el nuevo DataFrame a un archivo Excel
output_file_path = 'train_cyc_data_selected.xlsx'

# Usar la función to_excel para exportar el DataFrame
train_cyc_data_selected.to_excel(output_file_path, index=False)

print(f"El nuevo DataFrame ha sido exportado a {output_file_path}")


El nuevo DataFrame ha sido exportado a train_cyc_data_selected.xlsx


In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score

# Cargar el DataFrame refinado desde el archivo Excel
refined_data_path = 'train_cyc_data_selected.xlsx'
refined_data = pd.read_excel(refined_data_path)

# Separar las variables independientes (X) de la variable objetivo (y)
X = refined_data.drop(columns=['label'])
y = refined_data['label']

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=8, stratify=y)

# Inicializar modelos
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Random Forest": RandomForestClassifier(random_state=42),
    "Support Vector Classifier": SVC(probability=True, random_state=42),
    "K-Nearest Neighbors": KNeighborsClassifier()
}

# Evaluar cada modelo
results = {}
for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_pred_prob = model.predict_proba(X_test)[:, 1]
    
    # Calcular métricas de rendimiento
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred_prob)
    
    results[model_name] = {
        "Accuracy": accuracy,
        "F1 Score": f1,
        "ROC AUC Score": roc_auc
    }

# Convertir los resultados en un DataFrame para facilitar la visualización
results_df = pd.DataFrame(results).T

# Mostrar los resultados
print(results_df.sort_values(by="ROC AUC Score", ascending=False))


C:\Users\chris\miniconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


                           Accuracy  F1 Score  ROC AUC Score
Random Forest              0.948889  0.871985       0.988118
Logistic Regression        0.802222  0.356627       0.851065
Support Vector Classifier  0.793333  0.000000       0.802767
K-Nearest Neighbors        0.782222  0.363636       0.788828


In [17]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# Define the hyperparameters grid to search
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Initialize the Random Forest model
rf = RandomForestClassifier(random_state=42)

# Initialize Grid Search with cross-validation
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, 
                           cv=5, n_jobs=-1, verbose=2, scoring='roc_auc')

# Fit the model to the data
grid_search.fit(X_train, y_train)

# Retrieve the best parameters and best model
best_params = grid_search.best_params_
best_rf = grid_search.best_estimator_

print(f"Best parameters found: {best_params}")

# Evaluate the fine-tuned model on the test set
y_pred = best_rf.predict(X_test)
y_pred_prob = best_rf.predict_proba(X_test)[:, 1]

accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_prob)

print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print(f"ROC AUC Score: {roc_auc}")


Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Best parameters found: {'bootstrap': False, 'max_depth': 30, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 300}
Accuracy: 0.9607407407407408
F1 Score: 0.9027522935779817
ROC AUC Score: 0.9898630897998386


In [19]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

# Load your test data
test = pd.read_excel('Test.xlsx')

# Ensure the test data has the same features as the training data
# Step 1: Identify missing columns in test data and add them with default values (e.g., 0)
missing_cols = set(X_train.columns) - set(test.columns)
for col in missing_cols:
    test[col] = 0

# Step 2: Ensure columns are in the same order as in the training data
test = test[X_train.columns]

# Step 3: Apply the same scaling if required (example: for 'gross' and 'budget' columns)
scaler = StandardScaler()
test[['gross', 'budget']] = scaler.fit_transform(test[['gross', 'budget']])

# Initialize the Random Forest model with the best parameters
best_rf_model = RandomForestClassifier(
    bootstrap=False,
    max_depth=30,
    min_samples_leaf=2,
    min_samples_split=5,
    n_estimators=200,
    random_state=42
)

# Fit the model to your training data (assuming you have X_train and y_train already)
best_rf_model.fit(X_train, y_train)

# Make predictions on the test data
test_predictions = best_rf_model.predict(test)

# Optionally, get the probabilities for success
test_probabilities = best_rf_model.predict_proba(test)[:, 1]

# Add the predictions to the test DataFrame
test['Predicted_Success'] = test_predictions
test['Success_Probability'] = test_probabilities

# Save the results to a new Excel file or CSV
test.to_excel('predicted_test_results.xlsx', index=False)

# Print out the first few rows to check
print(test.head())


   imdb_score     gross  duration  title_year  genre_Biography    budget  \
0         5.4  0.273012      77.0      2002.0                0  0.622176   
1         5.5 -0.028754      91.0      2001.0                0 -0.079061   
2         5.1 -0.580768      93.0      2014.0                0 -0.215100   
3         6.0  0.243438     104.0      2006.0                0  0.376743   
4         7.2 -0.159323      97.0      1981.0                0 -0.149184   

   genre_Drama  genre_Sci-Fi  genre_Documentary  genre_Mystery  ...  \
0            0             0                  0              0  ...   
1            0             0                  0              0  ...   
2            0             0                  0              0  ...   
3            0             0                  0              0  ...   
4            0             0                  0              0  ...   

   genre_Fantasy  genre_Sport  genre_Horror  genre_Crime  genre_Film-Noir  \
0              0            0          